In [11]:
pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install jsonpath

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [14]:
import json
import os
import xmltodict
import re
import time
import pandas as pd
from jsonpath import jsonpath
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

In [15]:
# Initialize a dictionary to store the data
data = {
    "Questions": [],
    "Answers": [],
    "Focus": []
}

# Function to process an XML file
def processXmlFile(completePath):
    # Open the XML file
    with open(completePath) as f:
        # Read the contents of the file
        xmlstring = f.read()

        try:
            # Parse the XML string into a dictionary using xmltodict library
            dataDict = xmltodict.parse(xmlstring, xml_attribs=False)
            
            # Extract the QAPair and Focus information from the dictionary
            listOfQA = json.loads(json.dumps(jsonpath(dataDict, '$..' + "QAPair")[0]))
            focus = json.loads(json.dumps(jsonpath(dataDict, '$..' + "Focus")[0]))
        except Exception as e:
            # Handle exceptions, such as empty QAPair or Focus
            return

        # Check if there is only a single QA pair, and convert it to a list if needed
        if isinstance(listOfQA, dict):
            listOfQA = [listOfQA]
        
        # Process each QA pair
        for qaPair in listOfQA:
            try:
                # Clean up the answer text
                x = re.sub(' +', ' ', qaPair['Answer'])
                x = re.sub('Key Points', "", x)
                x = x.replace("\n", "").replace("-", "")
                
                # Append the processed data to the data dictionary
                data['Answers'].append(x)
                data['Questions'].append(qaPair['Question'])
                data['Focus'].append(focus)
            except:
                # Handle any exceptions that occur during processing
                return

In [16]:
# List of folders with empty answers
foldersWithEmptyAnswers = [
    "10_MPlus_ADAM_QA",
    "11_MPlusDrugs_QA",
    "12_MPlusHerbsSupplements_QA",
    "readme.txt",  # As it does not contain any QAs
    "QA-TestSet-LiveQA-Med-Qrels-2479-Answers",  # Will use it later,
    "LICENSE.txt"
]

# Base path for the folders
BASE_PATH = "/kaggle/input/medquad"

# Iterate over the folders in the base path
for folder in os.listdir(BASE_PATH):
    # Check if the folder is in the list of folders with empty answers
    if folder in foldersWithEmptyAnswers:
        # If the folder is in the list, skip it and continue with the next folder
        continue
    else:
        # If the folder is not in the list, process it
        print("Processing folder:", folder)
        start = time.time()

        # Iterate over the XML files in the current folder
        for xmlFileName in os.listdir(os.path.join(BASE_PATH, folder)):
            completePath = os.path.join(BASE_PATH, folder, xmlFileName)
            
            # Process the XML file
            processXmlFile(completePath)

        print("Took", time.time() - start)

Processing folder: 7_SeniorHealth_QA
Took 0.1831510066986084
Processing folder: 2_GARD_QA
Took 3.6362688541412354
Processing folder: 6_NINDS_QA
Took 0.37604403495788574
Processing folder: 4_MPlus_Health_Topics_QA
Took 1.1131412982940674
Processing folder: 3_GHR_QA
Took 1.8462533950805664
Processing folder: 9_CDC_QA
Took 0.09801745414733887
Processing folder: 5_NIDDK_QA
Took 0.41692543029785156
Processing folder: 1_CancerGov_QA
Took 0.3968656063079834
Processing folder: 8_NHLBI_QA_XML
Took 0.22964262962341309


In [18]:
# After creating the DataFrame
df = pd.DataFrame(data)
df.to_csv('processed_medquad.csv', index=False)